In [1]:
# Notebook to interact with OT2 pipetting robot

In [3]:
def min_max(spectra):
    scaler = sklearn.preprocessing.MinMaxScaler()
    return scaler.fit_transform(spectra.reshape(-1,1)).reshape(1,-1)[0]


def spectra_from_conc(action, dyes):
    """Transforms actions and input into output."""
    fract = normalize_to_fraction(action)
    spec = dyes[:,0]*fract[0]
    for i in range(1, dyes.shape[-1]):
        spec += dyes[:,i]*fract[i]
    spec = min_max(spec)
    return spec

def normalize_to_fraction(action):
    """Returns the fraction of total volume given the concentrations (action)."""
    total = np.sum(action)
    return action / total

class Environment(object):
    def __init__(self, target, dyes):
        # in future this will be basis spectra
        self.target = target
        self.dyes = dyes
        self.metric = []
   
    def get_cos_sim(self, target, y):
        return np.average(cosine_similarity(target.reshape(1, -1), Y=y.reshape(1, -1)).squeeze())    
    
    def sample(self, action):
        # put functins in utils python file
        result = spectra_from_conc(action, self.dyes)
        sim = self.get_cos_sim(self.target, result)
        mse = MSE(self.target, result)
        self.metric.append(sim)
        #return -mse
        return sim

In [4]:
sample_spectra = pd.read_excel('../../data/Sample_spectra.xlsx')
sample_spectra = np.asarray(sample_spectra)
web_spectra = pd.read_csv('../../data/web_spectra.csv').to_numpy()
wavelength = sample_spectra[:,0]
# spectra of base dyes
RED = sample_spectra[:,1]
GREEN = sample_spectra[:,2]
BLUE = sample_spectra[:,3]
SKY_BLUE = web_spectra[:,1]
ERYTHROSINE = web_spectra[:,2]
TATRIZINE = web_spectra[:,3]
BERRY = web_spectra[:,4]
SUNFLOWER = web_spectra[:,5]
FAST_GREEN = web_spectra[:,6]
ALLURA_RED = web_spectra[:,7]
SUNSET_YELLOW = web_spectra[:,8]
BRILLIANT_BLUE = web_spectra[:,9]
INDIGO_CARMINE = web_spectra[:,10]

dyes = sample_spectra[:,1:4]
dyes = np.hstack((dyes, web_spectra[:,1:]))[:,0:7]

PURPLE = []
for i in range(len(BLUE)):
    if i>30:
        PURPLE.append(1.5)
    else:
        PURPLE.append(0)

PURPLE=np.array(PURPLE)

target_conc = np.random.dirichlet((1, 1, 1, 1, 1, 1, 1, 1), 1)[0]
target_spectra = spectra_from_conc(target_conc, dyes)
#print(target_spectra)
#target_spectra = np.array([i*(random.random()/2 + .5) for i in target_spectra])

target_spectra = []
for i in range(len(BLUE)):
    if i<5 or i>35:
        target_spectra.append(0)
    elif i<15 or i>25:
        target_spectra.append(1)
    else:
        target_spectra.append(1.8)
target_spectra = np.array(target_spectra)

target = target_spectra
    
env = Environment(target_spectra, dyes)

batch_size = 15
epochs = 20

In [5]:
#constraints
min_conc = 0.05
max_conc = 1

# parameter space
N = 5 # grid size

# construct param space
coeffs = np.linspace(min_conc, max_conc, N)
param_space = np.meshgrid(coeffs, coeffs, coeffs, coeffs, coeffs, coeffs, coeffs)

In [6]:
class BatchGPUCBv3(ucb.BatchGPUCBv2):
    """
    Batched Guassian Process Upper Confidence Bound agent V2. With
    GP regressions in-batch. Slower compute time, but should
    in general converge in fewer batchs than V1.
    """
    def __init__(self, *args, **kwargs):
        super(BatchGPUCBv3, self).__init__(*args, **kwargs)
        
    def batch_sample(self, xs):
        """
        Forgots assumed samples from within batch. Then samples the sets of
        input in xs using the environment object.
        Save each input and output pair to the X and Y attributes,
        respectively. Resets to_exclude for the next batch.

        Arguments:
            xs - The set of input parameters to sample.
                type == list or array of tuples
        """
        self.X = self.X[0:-self.batch_size]
        self.Y = self.Y[0:-self.batch_size]
        ys = []
        for x in xs:
            y = self.environment.sample(x)
            ys.append(y)
            self.X.append(x)
            self.Y.append(y)
        self.to_exclude = []
        
        # Prune 0 potential points in parameter space
        # If the best lcb is higher than a points ucb, we
        # no longer need to regress over that point
        # But don't want to remove information so keep points
        # In the space that we have sampled at
        threshold = np.max(self.mu - self.sigma)
        args = np.argwhere((self.mu+self.sigma) >= threshold)
        self.X_grid = self.X_grid[args.squeeze()]
        if len(self.X_grid) < self.batch_size:
            self.batch_size = len(self.X_grid)

        print(len(args))
        return None
    
    
    def learn(self):
        """
        Learning function.

        Each time learn is called, the agent
            1. Finds <batch_size> best samples, performing a new GP Regression
               after each sample, assuming that sample returns its mean
            2. "Forgots" assumed samples and actually samples those paramaters
               to get their corresponding batched outputs.
            3. Trains a new Guassian Process regressor on all data points it
               has seen, including the latest batched sampling.
            4. Saves the new predicted means and standard deviations.
            5. Increases the time step.

        For the first timestep, a london hypercube sampling method is used
        """
        if self.T == 0:
            start = time.time()
            self.london_hypercube_sample()
            print('LH', time.time() - start)
        else:
            best_idxs = []
            start1 = time.time()
            for i in range(self.batch_size):  # 1
                start2 = time.time()
                best_idx = self.get_best_ucb()
                best_idxs.append(best_idx.item())
                print('get_best_ucb', time.time() - start2)
                gp = sklearn.gaussian_process.GaussianProcessRegressor()
                start2 = time.time()
                self.false_sample(best_idx)
                X = np.array(self.X).reshape(-1, self.input_dimension)
                Y = np.array(self.Y).reshape(-1)
                gp.fit(X, Y)
                print('false_sample + fit', time.time() - start2)
                start2 = time.time()
                self.mu, self.sigma = gp.predict(self.X_grid, return_std=True)
                print('predict', time.time() - start2)

            print('v2 loop', time.time() - start1)
            start1 = time.time()
            self.batch_sample(self.X_grid[best_idxs])  # 2
            print('batch_sample', time.time() - start1)

        start1 = time.time()
        gp = sklearn.gaussian_process.GaussianProcessRegressor()  # 3
        X = np.array(self.X).reshape(-1, self.input_dimension)
        Y = np.array(self.Y).reshape(-1)
        gp.fit(X, Y)
        if len(self.X_grid.shape) == 1:
            self.X_grid = self.X_grid.reshape(1, -1)
        self.mu, self.sigma = gp.predict(self.X_grid, return_std=True)  # 4
        self.T = self.T + 1   # 5
        print('final step', time.time() - start1)

        return None
    
    
    
    
    
    def get_best_ucb(self):
        """
        Return the index of the batch with the highest UCB. That
        has not already been selected in that batch
        """
        argsort_arr = np.flip(np.argsort(self.mu + self.sigma *
                                         np.sqrt(self.beta)))
        i = 0
        idx = argsort_arr[i]
        while idx in self.to_exclude:
            i += 1
            if i >= len(argsort_arr):
                break
            idx = argsort_arr[i]
        self.to_exclude.append(idx)
        return idx


# Initialize agent

In [7]:
agent = BatchGPUCBv3(batch_size, param_space, env, beta=3)
agent1 = ucb.BatchGPUCBv2(batch_size, param_space, env, beta=3)
agent2 = ucb.BatchGPUCB(batch_size, param_space, env, beta=3)


# Learn

In [8]:
# training loop
start1 = time.time()
for i in range(epochs):
    print('learning1')
    agent.learn()
    agent1.learn()
    agent2.learn()
    
t1 = time.time() - start1

learning1
78125
LH 0.06136608123779297
final step 0.11449193954467773
learning1
get_best_ucb 0.014452934265136719
false_sample + fit 0.002546072006225586
predict 0.06206226348876953
get_best_ucb 0.006910800933837891
false_sample + fit 0.0019528865814208984
predict 0.05998992919921875
get_best_ucb 0.007487058639526367
false_sample + fit 0.0024399757385253906
predict 0.12404298782348633
get_best_ucb 0.0307159423828125
false_sample + fit 0.004715919494628906
predict 0.16205406188964844
get_best_ucb 0.021123170852661133
false_sample + fit 0.009903907775878906
predict 0.13848304748535156
get_best_ucb 0.008545875549316406
false_sample + fit 0.001744985580444336
predict 0.08718705177307129
get_best_ucb 0.009542703628540039
false_sample + fit 0.009207963943481445
predict 0.11236691474914551
get_best_ucb 0.00852823257446289
false_sample + fit 0.0027170181274414062
predict 0.10299921035766602
get_best_ucb 0.00964212417602539
false_sample + fit 0.0016667842864990234
predict 0.08602285385131836
ge

KeyboardInterrupt: 

In [ ]:
class BatchGPUCB(ucb.GPUCB):
    """Batched Guassian Process Upper Confidence Bound agent V1."""

    def __init__(self, batch_size, *args, **kwargs):
        """
        Child init function.

        Arguments:
            batch_size - The number of trials in a single batch.
                type == int
        args:
            meshgrid - The parameter space on which to explore possible inputs.
                Expected to be the output from np.meshgrid
                type == list
            environment - Environment class. Should have a 'sample' function.
                type == class
        kwargs:
            beta - Hyper-parameter to tune the exploration-exploitation
                balance. If beta is large, it emphasizes the variance of the
                unexplored solution solution (i.e. larger curiosity)
                default = 100
                type == float
        """
        # setting batch size attribute
        self.batch_size = batch_size
        # inherent everything else from parent class
        super(BatchGPUCB, self).__init__(*args, **kwargs)

    def argsort_ucb(self):
        """Return the indices of the batch with the highest UCB."""
        argsort_arr = np.flip(np.argsort(self.mu + self.sigma *
                                         np.sqrt(self.beta)))
        return argsort_arr[:self.batch_size]

    def learn(self):
        """
        Learning function.

        Each time learn is called, the agent
            1. Finds the batch of input parameters with the highest upper
               confidence bound.
            2. Samples those paramaters to get their corresponding batched
               outputs.
            3. Trains a new Guassian Process regressor on all data points it
               has seen, including the latest batched sampling.
            4. Saves the new predicted means and standard deviations.
            5. Increases the time step.
        """
        if self.T == 0:
            self.london_hypercube_sample()
        else:
            grid_indices = self.argsort_ucb()  # 1
            self.batch_sample(self.X_grid[grid_indices])  # 2
        # get ucb's from GP
        gp = sklearn.gaussian_process.GaussianProcessRegressor()
        # reshape appropriately
        X = np.array(self.X).reshape(-1, self.input_dimension)
        Y = np.array(self.Y).reshape(-1)
        gp.fit(X, Y)  # 3
        self.mu, self.sigma = gp.predict(self.X_grid, return_std=True)  # 4
        # increase time step
        self.T = self.T + 1  # 5
        return None


    def batch_sample(self, xs):
        """
         Sample the sets of input in xs using the environment object.
         Save each input and output pair to the X and Y attributes,
         respectively.

         Arguments:
             xs - The set of input parameters to sample.
                 type == list or array of tuples
        """
        ys = []
        for x in xs:
            y = self.environment.sample(x)
            ys.append(y)
        self.Y.append(ys)
        self.X.append(xs)
        return None

    def london_hypercube_sample(self):
        """
        Does a London Hypercube (LH) sampling of the parameter space.
        LH based on the indexes of the parameter space. Use indices
        to get X values and pass into batch_sample.
        """

        limits = []
        for dim in self.meshgrid.shape[1:]:
            limits.append([0, dim-1])

        LH_sampler = smt.sampling_methods.LHS(xlimits=np.array(limits))
        sampled = np.round(LH_sampler(self.batch_size)).astype(int)
        t = self.meshgrid.T
        xs = [t[tuple(sample)] for sample in sampled]
        self.batch_sample(xs)
        return None

In [ ]:
agent2 = BatchGPUCB(batch_size, param_space, env, beta=3)
start2 = time.time()
for i in range(epochs):
    print('learning2')
    agent2.learn()
t2 = time.time() - start2

In [ ]:
print(t1)
print(t2)

In [ ]:
data = agent.X

In [ ]:
fig, axs = plt.subplots(nrows=epochs*3//2, ncols=2, figsize=(12, 30))
axs=axs.flatten()
for i in range(epochs*3):
    #plt.figure()
    axs[i].plot(min_max(target_spectra), color='black', label='target')
    textstr=f"batch {i+1}"
    axs[i].text(0.05, 0.95, textstr, transform=axs[i].transAxes, fontsize=14,
        verticalalignment='top')
    for j in range(batch_size):
        spec = spectra_from_conc(data[i*batch_size + j], dyes)
        axs[i].plot(spec, alpha=.5)
    
plt.setp(axs[[-2,-1]], xlabel='wavelength')
plt.setp(axs[::2], ylabel='absorption')

In [ ]:
agent.X[-batch_size:]

In [ ]:
data = agent2.X
fig, axs = plt.subplots(nrows=epochs//2, ncols=2, figsize=(12, 10))
axs=axs.flatten()
for i in range(epochs):
    #plt.figure()
    axs[i].plot(min_max(target_spectra), color='black', label='target')
    textstr=f"batch {i+1}"
    axs[i].text(0.05, 0.95, textstr, transform=axs[i].transAxes, fontsize=14,
        verticalalignment='top')
    for j in range(batch_size):
        spec = spectra_from_conc(data[i*batch_size + j], dyes)
        axs[i].plot(spec, alpha=.5)
    
plt.setp(axs[[-2,-1]], xlabel='wavelength')
plt.setp(axs[::2], ylabel='absorption')

In [ ]:
plt.figure()
plt.plot(wavelength, RED)
plt.plot(wavelength, GREEN)
plt.plot(wavelength, BLUE)
plt.plot(wavelength, SKY_BLUE)
plt.plot(wavelength, ERYTHROSINE)
plt.plot(wavelength, TATRIZINE)
plt.plot(wavelength, BERRY)


plt.plot(wavelength, SUNFLOWER)
plt.plot(wavelength, FAST_GREEN)

plt.plot(wavelength, ALLURA_RED)

plt.plot(wavelength, SUNSET_YELLOW)
plt.plot(wavelength, BRILLIANT_BLUE)
plt.plot(wavelength, INDIGO_CARMINE)


In [ ]:
mu = agent.mu
sigma = agent.sigma
threshold = np.max(mu - sigma)
X_grid = agent.X_grid

In [ ]:
args = np.argwhere((mu+sigma) < threshold)

In [ ]:
len(mu)

In [ ]:
len(mu[args])

In [ ]:
X_grid

In [ ]:
new = X_grid[args.squeeze()]

In [ ]:
new

In [ ]:
X_grid

In [ ]:
new.shape

In [ ]:
X_grid.shape